In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [ ]:
data_path=r'./dataset/researchDataset/DS07012.csv'
data = pd.read_csv(data_path, delimiter=',', index_col=False)
df = data.drop('Class', axis=1)
df.head()

In [ ]:
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X.shape

In [ ]:
k_best = SelectKBest(f_regression, k=40)

X_new = k_best.fit_transform(X, y)
feature_names = k_best.get_feature_names_out()

In [ ]:
selected_40_best = list(feature_names)
features_kept = ['Class'] + selected_40_best + ['Testability']
feature_selected_data = data.filter(features_kept)

new_data_path=r'./dataset/newDataset/DS_K40.csv'
feature_selected_data.to_csv(new_data_path, index=False)

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

min_features_to_select = 15  # Minimum number of features to consider
#estimator = LogisticRegression()
#estimator = RandomForestRegressor(n_estimators=100, n_jobs = -1)
estimator = SVR(kernel="linear")
#cv = KFold()

rfecv = RFECV(
    estimator=estimator,
    step=5,
    min_features_to_select=min_features_to_select,
    cv=3
)
rfecv.fit(X_new, y)

print(f"Optimal number of features: {rfecv.n_features_}")